**Lab-08: Backpropagation**

Trong bài thực hành này chúng ta sẽ thử cài đặt Backpropagation 

Ta muốn dựa vào 2 chiều của lá, phân biệt giữa loại lá 1 và loại lá 2. Cụ thể, với $x = (x_1,x_2, 1)$ là input, ta muốn đoán một phân phối
    $$ P_\\theta(c|x),c = 0, 1 $$
với $\\theta$ là các tham số
Ta mô hình $P_\theta$ là một neural network có 2 lớp ẩn, mỗi lớp 5 neurons, tức là\n",
    $$ P_\\theta(c|x) = \\text{softmax}(\\max(0, \\max(0, x \\cdot W_1 + b_1) \\cdot W_2 + b_2) \\cdot W_3 + b_3 )$$

với $x$ là vector dòng $[[x_1, x_2]]$ kích thước $ 1\times 2$, $W_1, W_2, W_3$ là các ma trận có kích thước $2 \times 5, 5 \times 5, 5 \times 3$, và $b_1, b_2, b_3$ là các ma trận kích thước $1 \times 5, 1 \times 5, 1 \times 3$.

Khi đó $P(c|x)$ là một vector dòng độ dài 3, xem như $P(c|x)= (P_1(c|x), P_2(c|x), P_3(c|x)) = (P(c=0|x), P(c=1|x), P(c=2|x))$
Bộ các ma trận $\\theta = (W_1, W_2, W_3, b_1, b_2, b_3)$ chính là tham số cần tìm của model. Giờ cần tìm $\\theta$ sao cho 

$$ L = \frac{1}{N} \sum_{x,y} - y_0 \log P_\theta(0|x) -  y_1 \log P_\theta(1|x) - y_2 \log P_\theta(2|x) $$

đạt giá trị nhỏ nhất với $y = (y_0, y_1, y_2)$ là one-hot vector biểu thị loại lá tương ứng với $x$



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Preprocessing

In [2]:
def one_hot_vector(y):
    out = np.zeros((y.shape[0], max(y)+1))
    for i in range(y.shape[0]):
        out[i, y[i]] = 1
    return out

In [3]:
train = pd.read_csv("https://raw.githubusercontent.com/huynhthanh98/ML/master/lab-08/bt_train.csv")
valid = pd.read_csv("https://raw.githubusercontent.com/huynhthanh98/ML/master/lab-08/bt_valid.csv")

x1_train = train["x1"].values
x2_train = train["x2"].values
y_train = train["label"].values

x1_valid = valid['x1'].values
x2_valid = valid['x2'].values
y_valid = valid['label'].values

# normalize
x1_mean = np.mean(x1_train)
x1_std = np.std(x1_train)
x2_mean = np.mean(x2_train)
x2_std = np.std(x2_train)

x1_train = (x1_train - x1_mean)/ x1_std
x2_train = (x2_train - x2_mean)/ x2_std

x1_valid = (x1_valid - x1_mean)/ x1_std
x2_valid = (x2_valid - x2_mean)/ x2_std



X_train = np.concatenate([x1_train.reshape(-1,1), x2_train.reshape(-1,1)], axis=1)
y_train = one_hot_vector(y_train)

X_valid = np.concatenate([x1_valid.reshape(-1,1), x2_valid.reshape(-1,1)], axis=1)

In [4]:
# initialize
W1 = np.random.randn(2,5)
W2 = np.random.randn(5,5)
W3 = np.random.randn(5,3)

b1 = np.random.randn(1,5)
b2 = np.random.randn(1,5)
b3 = np.random.randn(1,3)

In [5]:
def relu(h):
    return np.array([max(0,i) for i in h.reshape(-1)]).reshape(h.shape)

def softmax(z):
      return np.exp(z)/ np.sum(np.exp(z), axis=1).reshape(-1,1)

def CrossEntropy(o,y):
      return - np.sum(np.log(o)*y)

ln = 0.001
N = y_train.shape[0]

In [6]:
for epochs in range(100000):
    # foward
    z1 = np.dot(X_train, W1) + b1
    o1 = relu(z1)

    z2 = np.dot(o1, W2) + b2
    o2 = relu(z2)

    z3 = np.dot(o2, W3) + b3
    o3 = softmax(z3)

    # backpropagation
    dL_dz3 = 1/len(X_train)*(o3 - y_train) 
    dL_dW3 = np.dot(o2.T, dL_dz3)    
    dL_db3 = np.sum(dL_dz3, axis = 0)


    dL_do2 = np.dot(dL_dz3, W3.T)
    dL_dz2 = dL_do2.copy()
    dL_dz2[z2 < 0] = 0
    dL_dW2 = np.dot(o1.T,dL_dz2)
    dL_db2 = np.sum(dL_dz2, axis = 0)

    dL_do1 = np.dot(dL_dz2, W2.T)
    dL_dz1 = dL_do1.copy()
    dL_dz1[z1 < 0] = 0
    dL_dW1 = np.dot(X_train.T, dL_dz1)
    dL_db1 = np.sum(dL_dz1, axis = 0)

    W3 -= ln* dL_dW3
    b3 -= ln* dL_db3
    W2 -= ln* dL_dW2
    b2 -= ln* dL_db2
    W1 -= ln* dL_dW1
    b1 -= ln* dL_db1

In [7]:
z1_valid = np.dot(X_valid, W1) + b1
o1_valid = relu(z1_valid)

z2_valid = np.dot(o1_valid, W2) + b2
o2_valid = relu(z2_valid)

z3_valid = np.dot(o2_valid, W3) + b3
o3_valid = softmax(z3_valid)

In [8]:
np.sum(np.argmax(o3_valid, axis = 1) == y_valid)/ y_valid.shape[0]

0.6366666666666667

#Bài Tập
1. Từ code demo hãy cài đặt thêm một module để chọn ra được bộ weights sao cho accuracy trên tập validation là tốt nhất.
2. Từ bộ dữ liệu bên dưới hãy cài đặt backpropagation cho bài toán phân biệt ung thư vú. Hãy tự chọn số layers và số nodes mà mình cho là thích hợp, cũng như là nêu ra số layers và số nodes của mỗi layer mà mình đã chọn. Tính accuracy trên tập training.

Câu 1

In [9]:
# initialize
W1 = np.random.randn(2,5)
W2 = np.random.randn(5,5)
W3 = np.random.randn(5,3)

b1 = np.random.randn(1,5)
b2 = np.random.randn(1,5)
b3 = np.random.randn(1,3)

In [10]:
for epoch in range(100000):
    #forward
    z1 = np.dot(X_train, W1) + b1
    o1 = relu(z1)
    
    z2 = np.dot(o1, W2) + b2
    o2 = relu(z2)
    
    z3 = np.dot(o2, W3) + b3
    o3 = softmax(z3)
    
    #backpropagation
    dL_dz3 = 1/len(X_train)*(o3 - y_train)
    dL_dW3 = np.dot(o2.T, dL_dz3)
    dL_db3 = np.sum(dL_dz3, axis = 0)
    
    
    dL_do2 = np.dot(dL_dz3, W3.T)
    dL_dz2 = dL_do2.copy()
    dL_dz2[z2 < 0] = 0
    dL_db2 = np.sum(dL_dz2, axis = 0)
    
    dL_do1 = np.dot(dL_dz2, W2.T)
    dL_dz1 = dL_do1.copy()
    dL_dz1[z1 < 0] = 0
    dL_dW1 = np.dot(X_train.T, dL_dz1)
    dL_db1 = np.sum(dL_dz1, axis = 0)

    W3 -= ln* dL_dW3
    b3 -= ln* dL_db3
    W2 -= ln* dL_dW2
    b2 -= ln* dL_db2
    W1 -= ln* dL_dW1
    b1 -= ln* dL_db1

    z1_valid = np.dot(X_valid, W1) + b1
    o1_valid = relu(z1_valid)

    z2_valid = np.dot(o1_valid, W2) + b2
    o2_valid = relu(z2_valid)

    z3_valid = np.dot(o2_valid, W3) + b3
    o3_valid = softmax(z3_valid)

    valid_accuracy = np.sum(np.argmax(o3_valid, axis = 1) == y_valid)/ y_valid.shape[0]

    if valid_accuracy > max_valid_accuracy:
        max_valid_accuracy = valid_accuracy
        epoch_opti = epoch

        W3_opti = W3
        b3_opti = b3
        W2_opti  = W2
        b2_opti = b2
        W1_opti  = W1
        b1_opti = b1

    if epoch % 1000 == 0:
        print(f"Accuracy in validation set at epoch {epoch} is {valid_accuracy:.4f}")

print(f"Highest Accuracy in validation set obtained at epoch {epoch_opti} is {max_valid_accuracy:.4f}")
print(f"Optimal weight are storaged in Wx_opti and bx_opti")

NameError: name 'max_valid_accuracy' is not defined

Ta thấy Accuracy cao nhất đạt được là ở epoch cuối cùng trong quá trình train. 
Do đó có thể thấy là mô hình có khả năng cao là chưa bị overfitting. Có thể tăng số epoch train để mô hình hội tụ tốt hơn

Bộ parameter 𝑡ℎ𝑒𝑡𝑎=(𝑊1,𝑊2,𝑊3,𝑏1,𝑏2,𝑏3) được lưu lại khi mỗi epoch train trên tập training set mà có Accuracy tăng so với Accuracy cao nhất đã đạt được trước đó

Câu 2

In [ ]:
from sklearn import datasets

breast_cancer = datasets.load_breast_cancer()
X = breast_cancer.data  
y = breast_cancer.target

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split( X, y, test_size=0.2, random_state=42)

X_mean=np.mean(X_train)
X_std=np.std(X_train)

X_valid=(X_valid-X_mean)/X_std
X_train=(X_train-X_mean)/X_std

In [ ]:
#Sử dụng one hot vector cho y_train để tính Loss với hàm softmax ở layer cuối
y_train_1hot = one_hot_vector(y_train)

In [ ]:
print(list(breast_cancer.feature_names))
print(f'Số lượng features input: {len(list(breast_cancer.feature_names))}')

print(f'Số lượng classes output: {len(list(np.unique(y)))}')

Bộ dữ liệu về ung thư vú có 30 features ứng với 30 nodes input, 2 nodes output

Ta thiết kế mạng Neuron với 3 hidden layer với số nodes lần  lượt là <br>
Lớp 1: 15 node, Lớp 2: 15 node, Lớp 3: 5 node


Kích thước các ma trận W1, W2, W3, W4 tương ứng là: 30x15, 15x15, 15x5, 5x2 <br>
Kích thước các ma trận W1, W2, W3, W4 là: 1x15, 1x15, 1x5, 1x2

In [ ]:
# initialize
W1 = np.random.randn(30,15)
W2 = np.random.randn(15,15)
W3 = np.random.randn(15,5)
W4 = np.random.randn(5,2)

b1 = np.random.randn(1,15)
b2 = np.random.randn(1,15)
b3 = np.random.randn(1,5)
b4 = np.random.randn(1,2)

ln = 0.01

In [ ]:
max_valid_accuracy = 0

for epoch in range(100000):
    #foward
    z1 = np.dot(X_train, W1) + b1
    o1 = relu(z1)

    z2 = np.dot(o1, W2) + b2
    o2 = relu(z2)

    z3 = np.dot(o2, W3) + b3
    o3 = relu(z3)

    z4 = np.dot(o3, W4) + b4
    o4 = softmax(z4)

    #backpropagation
    dL_dz4 = 1/len(X_train)*(o4 - y_train_1hot) 
    dL_dW4 = np.dot(o3.T, dL_dz4)    
    dL_db4 = np.sum(dL_dz4, axis = 0)

    dL_do3 = np.dot(dL_dz4, W4.T)
    dL_dz3 = dL_do3.copy()
    dL_dz3[z3 < 0] = 0
    dL_dW3 = np.dot(o2.T,dL_dz3)
    dL_db3 = np.sum(dL_dz3, axis = 0)

    dL_do2 = np.dot(dL_dz3, W3.T)
    dL_dz2 = dL_do2.copy()
    dL_dz2[z2 < 0] = 0
    dL_dW2 = np.dot(o1.T,dL_dz2)
    dL_db2 = np.sum(dL_dz2, axis = 0)

    dL_do1 = np.dot(dL_dz2, W2.T)
    dL_dz1 = dL_do1.copy()
    dL_dz1[z1 < 0] = 0
    dL_dW1 = np.dot(X_train.T, dL_dz1)
    dL_db1 = np.sum(dL_dz1, axis = 0)

    W4 -= ln* dL_dW4
    b4 -= ln* dL_db4
    W3 -= ln* dL_dW3
    b3 -= ln* dL_db3
    W2 -= ln* dL_dW2
    b2 -= ln* dL_db2
    W1 -= ln* dL_dW1
    b1 -= ln* dL_db1

    z1_valid = np.dot(X_valid, W1) + b1
    o1_valid = relu(z1_valid)

    z2_valid = np.dot(o1_valid, W2) + b2
    o2_valid = relu(z2_valid)

    z3_valid = np.dot(o2_valid, W3) + b3
    o3_valid = softmax(z3_valid)

    z4_valid = np.dot(o3_valid, W4) + b4
    o4_valid = softmax(z4_valid)

    valid_accuracy = np.sum(np.argmax(o4_valid, axis = 1) == y_valid)/ y_valid.shape[0]

    if valid_accuracy > max_valid_accuracy:
        max_valid_accuracy = valid_accuracy
        epoch_opti = epoch

        W4_opti = W4
        b4_opti = b4
        W3_opti = W3
        b3_opti = b3
        W2_opti  = W2
        b2_opti = b2
        W1_opti  = W1
        b1_opti = b1

    if epoch % 1000 == 0:
        print(f"Accuracy in validation set at epoch {epoch} is {valid_accuracy:.4f}")

print(f"Highest Accuracy in validation set obtained at epoch {epoch_opti} is {max_valid_accuracy:.4f}")

In [ ]:
#Sử dụng bộ Parameters tốt được đã thu được sau khi train 10000 epochs
#Tính toán quá trình forward với input là bộ data train
z1_train = np.dot(X_train, W1_opti) + b1_opti
o1_train = relu(z1_train)

z2_train = np.dot(o1_train, W2_opti) + b2_opti
o2_train = relu(z2_train)

z3_train = np.dot(o2_train, W3_opti) + b3_opti
o3_train = softmax(z3_train)

z4_train = np.dot(o3_train, W4_opti) + b4_opti
o4_train = softmax(z4_train)

#So sánh Label tập train và Kết quả quá trình forward trên mạng Neuron 
np.sum(np.argmax(o4_train, axis = 1) == y_train)/ y_train.shape[0]